In [1]:
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, HiveContext
import os
from pyspark.sql.functions import *
import pyspark.storagelevel 
import numpy as np
import pandas as pd
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pdb
import subprocess # Used for executing linux commands, for writing to teradata
import sys

from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, min, max
from datetime import datetime, timedelta

from_jupyter = False

if "sc"  in globals():
        SparkContext.stop(sc)
        from_jupyter = True
                
SparkContext.setSystemProperty('spark.yarn.queue','gg-uk-tescoglobal-hadoop-rdf')
SparkContext.setSystemProperty('spark.executor.instances', '50')
SparkContext.setSystemProperty('spark.executor.memory', '5g')
#SparkContext.setSystemProperty('spark.yarn.executor.memoryOverhead','10000')
SparkContext.setSystemProperty('spark.executor.cores', '1')
SparkContext.setSystemProperty('spark.sql.shuffle.partitions', '10000')
SparkContext.setSystemProperty('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
SparkContext.setSystemProperty('spark.default.parallelism','2000')
#SparkContext.setSystemProperty('spark.driver.maxResultSize','20g')
SparkContext.setSystemProperty('spark.executor.heartbeatInterval','60s')

if from_jupyter:
    sc = SparkContext("yarn-client", "Mid term Forecast")
else:
    conf = SparkConf().setAppName("Mid term Forecast")
    sc = SparkContext(conf=conf)

sqlContext = SQLContext(sc)
sqlContext = HiveContext(sc)

In [2]:
print datetime.now()

2017-10-16 05:56:52.128644


In [3]:
# Parameters that are required as prerequisites
forecast_start_day = 0
forecast_end_day = 91

# current_date = '2017-08-16'
# forecast_start = '2017-08-17'
# forecast_end = '2017-11-15'
# cal_fcst_start = '20170817'
# cal_fcst_end = '20171115'
# sub_group = 'allSG'

current_date = (datetime.now()+ timedelta(days=-1)).strftime("%Y-%m-%d")

forecast_start=(datetime.now() + timedelta(days=forecast_start_day)).strftime("%Y-%m-%d")
forecast_end=(datetime.now() + timedelta(days=forecast_end_day)).strftime("%Y-%m-%d")

forecast_start_date=(datetime.now() + timedelta(days=forecast_start_day)).date()
forecast_end_date=(datetime.now() + timedelta(days=forecast_end_day)).date()

cal_fcst_start = (datetime.now() + timedelta(days=forecast_start_day)).strftime("%Y%m%d")
cal_fcst_end = (datetime.now() + timedelta(days=forecast_end_day)).strftime("%Y%m%d")
sub_group='F11MA'

print current_date, forecast_start, forecast_end, cal_fcst_start, cal_fcst_end

2017-10-15 2017-10-16 2018-01-15 20171016 20180115


In [4]:

# Lets get the CAL for the date range that we are interested in.
query = "(select * from DXWI_PROD_VIEW_ACCESS.VWI0CAL_CALENDAR \
          where calendar_date between '{0}' and '{1}') as CAL_DATA order by CAL_DATA.Calendar_Date" \
         .format(cal_fcst_start, cal_fcst_end)

cal_data = (sqlContext.read.format('jdbc')
                .options(url="jdbc:teradata://TDPS.ukroi.tesco.org/",
                     dbtable=query,
                     user = "xg66",
                     password = "tesco123",
                     driver = "com.teradata.jdbc.TeraDriver")
                .option("treatEmptyValuesAsNulls","true")
                .option("inferSchema","false")
               # .option("nullValues","null")
                .load()
                .cache()
     )

In [5]:
type(cal_data)

pyspark.sql.dataframe.DataFrame

In [6]:
cal_data.show(2)

+-------------+----------------+-----------+--------------+-------------+-----------+----------+---------+------------------+--------+--------+-------------------+-------------------+---------------------+
|Calendar_Date|Year_Week_Number|Year_Number|Quarter_Number|Period_Number|Week_Number|Day_Number| Day_Text|Period_Week_Number|Tesco_WF|Tesco_PF|Commercial_Tesco_WF|Commercial_Tesco_PF|Commercial_Day_Number|
+-------------+----------------+-----------+--------------+-------------+-----------+----------+---------+------------------+--------+--------+-------------------+-------------------+---------------------+
|   2017-10-16|          201734|       2017|             3|            8|         34|         1|MONDAY   |                 3| 2017341|20170831|            2017342|           20170832|                    2|
|   2017-10-17|          201734|       2017|             3|            8|         34|         2|TUESDAY  |                 3| 2017342|20170832|            2017343|           20

In [7]:
cal_sel_data = cal_data.select(col("CALENDAR_DATE").alias("calendar_date").cast(DateType()),
                               col("YEAR_WEEK_NUMBER").alias("year_week").cast(StringType()),
                               col("WEEK_NUMBER").alias("week_number").cast(StringType()),
                               col("DAY_NUMBER").alias("day_number").cast(StringType()))

In [8]:
cal_sel_data.count()

92

In [9]:
cal_fcst_start_year_week  =  cal_sel_data.select(min("year_week"),max("year_week")).collect()[0]["min(year_week)"]
cal_fcst_end_year_week  = cal_sel_data.select(min("year_week"),max("year_week")).collect()[0]["max(year_week)"]

In [10]:
cal_sel_data.select(min("year_week"),max("year_week")).collect()[0]["min(year_week)"]
#cal_sel_data.select(min("year_week"),max("year_week")).collect()[0]["max(year_week)"]

u'201734'

In [11]:
# cal_list = cal_sel_data.rdd.map(lambda x : (x.calendar_date,(x.year_week,x.week_number,x.day_number))).collectAsMap()
# cal_bc_var = sc.broadcast(cal_list)
# current_date_bc_var = sc.broadcast(current_date)
##vj_remove cal_sel_data.write.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/cal_bc_set1_'+sub_group,mode='overwrite')

In [12]:
# Input for F11MA sub group
sub_group='F11MA'
nws_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/cr_forecast/nws')
rrt_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/cr_forecast/rrt')
rst_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/cr_forecast/rst')
dst_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/cr_forecast/dst')
dtp_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/cr_forecast/ptp')
ptg_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/cr_forecast/ptg')
swf_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/cr_forecast/swf')
wef_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/cr_forecast/wef')

# Input for all sub groups
#sub_group = 'allSG'
#nws_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/golden_input/nws')
#rrt_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/golden_input/rrt')
#rst_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/golden_input/rst')
#dst_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/golden_input/dst')
#dtp_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/golden_input/ptp')
#ptg_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/golden_input/ptg')
#swf_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/golden_input/swf')
#wef_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/source_image/golden_input/wef')

In [13]:
nws_sel_data = nws_data.select(col("RETAIL_OUTLET_NUMBER").alias("ro_no").cast(IntegerType()),
                               col("BASE_PRODUCT_NUMBER").alias("bpr_tpn").cast(IntegerType()),
                               col("SUB_GROUP_CODE").alias("sg_cd").cast(StringType()),
                               col("NWS_STAT").alias("nws_stat").cast(StringType()),
                               col("STEP_IND").alias("step_ind").cast(StringType()),
                               col("FINAL_NWS").alias("final_nws").cast(DoubleType()),
                               col("SAVED_FINAL_NWS").alias("saved_final_nws").cast(DoubleType()),
                               col("AGGR_GRP_ID").alias("aggr_grp_id").cast(StringType()),
                               col("STOCKED_PROD_START_DATE").alias("stkd_prod_stdt").cast(DateType()),
                               col("STOCKED_PROD_END_DATE").alias("stkd_prod_endt").cast(DateType()))

In [ ]:
nws_schema

In [ ]:
nws_merged_rdd.take(2)

In [14]:
if nws_sel_data.rdd.getNumPartitions() < 2000:
            nws_sel_data  = nws_sel_data.repartition(2000).cache()

nws_schema = StructType([StructField("nws_key",StructType([StructField("ro_no",IntegerType(),True),
                                                           StructField("bpr_tpn",IntegerType(),True)]),True),
                         StructField("nws_merged_values",
                                              ArrayType(StructType([StructField("sg_cd",StringType(),True),
                                                                    StructField("nws_stat",StringType(),True),
                                                                    StructField("step_ind",StringType(),True),
                                                                    StructField("final_nws",DoubleType(),True),
                                                                    StructField("saved_final_nws",DoubleType(),True),
                                                                    StructField("aggr_grp_id",StringType(),True),
                                                                    StructField("stkd_prod_stdt",DateType(),True),
                                                                    StructField("stkd_prod_endt",DateType(),True)]),True),
                                     True)
                        ]
                       )
nws_merged_rdd = (nws_sel_data
                         .withColumn("key",struct([col("ro_no"),
                                                   col("bpr_tpn")
                                                  ]))
                         .withColumn("value",array([struct([
                                                           col("sg_cd"),
                                                           col("nws_stat"),
                                                           col("step_ind"),
                                                           col("final_nws"),
                                                           col("saved_final_nws"),
                                                           col("aggr_grp_id"),
                                                           col("stkd_prod_stdt"),
                                                           col("stkd_prod_endt")
                                                           ])
                                                 ])
                                    )
                         .select("key","value")
                         .rdd
                         .reduceByKey(lambda v1,v2 : v1 + v2)
                         #.toDF(["nws_key","nws_merged_values"])
                )

nws_merged_df = sqlContext.createDataFrame(nws_merged_rdd,nws_schema)

In [15]:
nws_merged_df.count()

51666

In [16]:
nws_merged_df.show(5)

+---------------+--------------------+
|        nws_key|   nws_merged_values|
+---------------+--------------------+
|[2920,81003974]|[[F11BD,W,Y,381.9...|
|[5700,54550994]|[[F51CA,A,N,968.1...|
|[2586,52466256]|[[F51CA,A,N,279.9...|
|[3190,50962236]|[[F11MA,W,Y,86.06...|
|[6807,54550971]|[[F51CA,A,N,84.87...|
+---------------+--------------------+
only showing top 5 rows



In [17]:
rrt_sel_data = rrt_data.select(col("RETAIL_OUTLET_NUMBER").alias("ro_no").cast(IntegerType()),
                               col("BASE_PRODUCT_NUMBER").alias("bpr_tpn").cast(IntegerType()),
                               col("STEP_EFF_DT").alias("rrt_step_eff_dt").cast(DateType()),
                               col("STEP_TYPE_NUM").alias("rrt_step_type_num").cast(IntegerType()),
                               col("STEP_UPLIFT_PC").alias("rrt_step_uplift").cast(IntegerType()),
                               col("STEP_VOLUME").alias("rrt_step_volume").cast(DoubleType()))

In [78]:
if rrt_sel_data.rdd.isEmpty():
    schema=StructType([StructField("rrt_ro_no", IntegerType(), True), StructField("rrt_bpr_tpn", IntegerType(), True), StructField("mergedInput", StringType(), True)])
    rrt_merged_df = sqlContext.createDataFrame(sc.parallelize([(0,0,'0')]),schema=schema)
    rrt_merged_rdd = rrt_merged_df.rdd.map(lambda row : ((row.rrt_ro_no,row.rrt_bpr_tpn),row.mergedInput))
    rrt_merged_df = rrt_merged_rdd.toDF(["rrt_key","rrt_merged_values"])
else:
    
    if rrt_sel_data.rdd.getNumPartitions() < 400:
            rrt_sel_data  = rrt_sel_data.repartition(400)
   
    
    rrt_schema = StructType([StructField("rrt_key",StructType([StructField("ro_no",IntegerType(),True),
                                                               StructField("bpr_tpn",IntegerType(),True)]),True),
                             StructField("rrt_merged_values",
                                         ArrayType(StructType([StructField("rrt_step_eff_dt",DateType(),True),
                                                               StructField("rrt_step_type_num",IntegerType(),True),
                                                               StructField("rrt_step_uplift",IntegerType(),True),
                                                               StructField("rrt_step_volume",DoubleType(),True)]),True),
                                     True)
                        ]
                       )
    
    rrt_merged_rdd = (rrt_sel_data
                         .withColumn("key",struct([col("ro_no"),
                                                   col("bpr_tpn")]))
                         .withColumn("value",array([struct([
                                                           col("rrt_step_eff_dt"),
                                                           col("rrt_step_type_num"),
                                                           col("rrt_step_uplift"),
                                                           col("rrt_step_volume")
                                                           ])
                                                 ])
                                    )
                         .select("key","value")
                         .rdd
                         .reduceByKey(lambda v1,v2 : v1 + v2)
                         #.toDF(["rrt_key","rrt_merged_values"])
                        )
    rrt_merged_df = sqlContext.createDataFrame(rrt_merged_rdd,rrt_schema)
    
rrt_merged_df.show(3)

+---------------+--------------------+
|        rrt_key|   rrt_merged_values|
+---------------+--------------------+
|[2468,54739758]|[[2017-10-17,251,...|
|[2364,57755937]|[[2017-10-15,251,...|
|[2927,50502269]|[[2017-10-15,351,...|
+---------------+--------------------+
only showing top 3 rows



In [19]:
rrt_merged_df.count()

69

In [20]:
join_cond1 = [nws_merged_df.nws_key == rrt_merged_df.rrt_key]
nws_rrt_df = nws_merged_df.join(rrt_merged_df,join_cond1,"left_outer").drop("rrt_key")#.cache()

In [21]:
#rst_data = rst_data.repartition(1000)
rst_sel_data = rst_data.select(col("RETAIL_OUTLET_NUMBER").alias("ro_no").cast(IntegerType()),
                               col("BASE_PRODUCT_NUMBER").alias("bpr_tpn").cast(IntegerType()),
                               col("STEP_EFF_DT").alias("rst_step_eff_dt").cast(DateType()),
                               col("STEP_END_DT").alias("rst_step_end_dt").cast(DateType()),
                               col("STEP_TYPE_NUM").alias("rst_step_type_num").cast(IntegerType()),
                               col("STEP_ID").alias("rst_step_id").cast(IntegerType()),
                               col("UNDERLYING_STEP_ID").alias("rst_underlying_step_id").cast(IntegerType()),
                               col("STEP_UPLIFT_PC").alias("rst_step_uplift").cast(IntegerType()),
                               col("STEP_VOLUME").alias("rst_step_volume").cast(DoubleType()),
                               col("ACTUAL_NWS").alias("rst_actual_nws").cast(DoubleType()),
                               col("WTHR_SENS_FLAG").alias("rst_wthr_sens_flag").cast(StringType()),
                               col("CONTINUE_NWS").alias("rst_continue_nws").cast(StringType())
                              )
#rst_sel_data.filter(rst_sel_data.ro_no==5288).filter(rst_sel_data.bpr_tpn==50935010).show()

In [22]:
rst_sel_data.show(2)

+-----+--------+---------------+---------------+-----------------+-----------+----------------------+---------------+---------------+--------------+------------------+----------------+
|ro_no| bpr_tpn|rst_step_eff_dt|rst_step_end_dt|rst_step_type_num|rst_step_id|rst_underlying_step_id|rst_step_uplift|rst_step_volume|rst_actual_nws|rst_wthr_sens_flag|rst_continue_nws|
+-----+--------+---------------+---------------+-----------------+-----------+----------------------+---------------+---------------+--------------+------------------+----------------+
| 6158|54550994|     2017-10-13|     2017-10-23|              241|  113100467|                     0|              0|           null|        425.52|                 Y|               N|
| 6545|61205253|     2017-10-16|     2017-10-16|              241|  114212202|                     0|            -10|           null|          null|                 Y|               N|
+-----+--------+---------------+---------------+-----------------+---------

In [23]:
# We need the minimum of step start and maximum of step end so that the same can be used to get the CAL range
rst_fcst_start = str(rst_sel_data.select(min("rst_step_eff_dt")).flatMap(lambda x:x).collect()[0]).replace('-','')
rst_fcst_end = str(rst_sel_data.select(max("rst_step_end_dt")).flatMap(lambda x:x).collect()[0]).replace('-','')

if rst_fcst_start > cal_fcst_start:
    rst_fcst_start = cal_fcst_start
    
if rst_fcst_end < cal_fcst_end:
    rst_fcst_end = cal_fcst_end
    
print rst_fcst_start, rst_fcst_end

20170809 20190915


In [24]:
# Let's get the forecast range interms of year week from CAL for the range of dates as obtained above. 
# This is needed to get only required PTP/PTG/SWF
query = "(select * from DXWI_PROD_VIEW_ACCESS.VWI0CAL_CALENDAR \
          where calendar_date between '{0}' and '{1}') as CAL_DATA order by CAL_DATA.Calendar_Date" \
         .format(rst_fcst_start, rst_fcst_end)

cal_data = (sqlContext.read.format('jdbc')
                .options(url="jdbc:teradata://TDPS.ukroi.tesco.org/",
                     dbtable=query,
                     user = "xg66",
                     password = "tesco123",
                     driver = "com.teradata.jdbc.TeraDriver")
                .option("treatEmptyValuesAsNulls","true")
                .option("inferSchema","false")
               # .option("nullValues","null")
                .load()
                .cache()
     )

In [25]:
cal_sel_data = cal_data.select(col("CALENDAR_DATE").alias("calendar_date").cast(DateType()),
                               col("YEAR_WEEK_NUMBER").alias("year_week").cast(StringType()),
                               col("WEEK_NUMBER").alias("week_number").cast(StringType()),
                               col("DAY_NUMBER").alias("day_number").cast(StringType()))
#cal_sel_data.show()

In [26]:

# cal_list = cal_sel_data.rdd.map(lambda x : (x.calendar_date,(x.year_week,x.week_number,x.day_number))).collectAsMap()
# cal_bc_var2 = sc.broadcast(cal_list)
#vijay cal_sel_data.write.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/cal_bc_set2_'+sub_group,mode='overwrite')

In [27]:
cal_yrwk_start = str(cal_data.select(min("Year_Week_Number")).flatMap(lambda x:x).collect()[0])
cal_yrwk_end = str(cal_data.select(max("Year_Week_Number")).flatMap(lambda x:x).collect()[0])

#print cal_yrwk_start, cal_yrwk_end

In [79]:
# Identify the minimum step eff date and maximum step end date for every store product
rst_sel_data_groupBy = (rst_sel_data.groupBy("ro_no","bpr_tpn")
                        .agg({"rst_step_eff_dt":"min","rst_step_end_dt":"max"})
                        .withColumnRenamed("min(rst_step_eff_dt)","min")
                        .withColumnRenamed("max(rst_step_end_dt)","max"))
rst_sel_data_groupBy.show(4)

+-----+--------+----------+----------+
|ro_no| bpr_tpn|       max|       min|
+-----+--------+----------+----------+
| 2894|50502269|2017-11-07|2017-10-18|
| 2848|81003974|2017-10-16|2017-10-16|
| 2388|54550994|2017-10-30|2017-10-13|
| 6849|57755937|2017-10-24|2017-10-13|
+-----+--------+----------+----------+
only showing top 4 rows



In [29]:
# Compare the max and min dates with the forecast range using UDF
min_start_date = udf(lambda x : x if x < forecast_start_date else forecast_start_date, DateType())
max_end_date = udf(lambda x : x if x > forecast_end_date else forecast_end_date, DateType())

rst_sel_data_groupBy_2 = rst_sel_data_groupBy.select("*",
                                                     min_start_date(col("min")).alias("min_fcst_start_date"),
                                                     max_end_date(col("max")).alias("max_fcst_end_date"))

#rst_sel_data_groupBy_2.show()

In [80]:
# Join the above with CAL table to arrive at the year week format for min_fcst_start_date and max_fcst_end_date
rst_sel_data_groupBy_3 = (rst_sel_data_groupBy_2.join(cal_sel_data,
                                                    rst_sel_data_groupBy_2["min_fcst_start_date"] == cal_sel_data.calendar_date,
                                                     "left_outer")
                                                    .select("ro_no"
                                                           ,"bpr_tpn"
                                                           ,"min_fcst_start_date"
                                                           ,"max_fcst_end_date"
                                                           ,"year_week")
                                                    .withColumnRenamed("year_week","min_fcst_start_year_week")
                                                .join(cal_sel_data,
                                                     rst_sel_data_groupBy_2["max_fcst_end_date"] == cal_sel_data.calendar_date,
                                                     "left_outer")
                                                    .select("ro_no"
                                                           ,"bpr_tpn"
                                                           ,"min_fcst_start_date"
                                                           ,"max_fcst_end_date"
                                                           ,"min_fcst_start_year_week"
                                                           ,"year_week")
                                                    .withColumnRenamed("year_week","max_fcst_end_year_week"))

rst_sel_data_groupBy_3.show(3)

+-----+--------+-------------------+-----------------+------------------------+----------------------+
|ro_no| bpr_tpn|min_fcst_start_date|max_fcst_end_date|min_fcst_start_year_week|max_fcst_end_year_week|
+-----+--------+-------------------+-----------------+------------------------+----------------------+
| 2894|50502269|         2017-10-16|       2018-01-15|                  201734|                  null|
| 2388|54550994|         2017-10-13|       2018-01-15|                  201733|                  null|
| 6849|57755937|         2017-10-13|       2018-01-15|                  201733|                  null|
+-----+--------+-------------------+-----------------+------------------------+----------------------+
only showing top 3 rows



In [31]:
if rst_sel_data.rdd.isEmpty():
    schema=StructType([StructField("ro_no", IntegerType(), True), StructField("bpr_tpn", IntegerType(), True), StructField("mergedInput", StringType(), True)])
    rst_merged_df = sqlContext.createDataFrame(sc.parallelize([(0,0,'0')]),schema=schema)
    rst_merged_rdd = rst_merged_df.rdd.map(lambda row : ((row.rst_ro_no,row.rst_bpr_tpn),row.mergedInput))
    rst_merged_df = rst_merged_rdd.toDF(["rst_key","rst_merged_values"])
else:
    
    if rst_sel_data.rdd.getNumPartitions() < 400:
            rst_sel_data  = rst_sel_data.repartition(400)
   
    rst_schema = StructType([StructField("rst_key",StructType([StructField("ro_no",IntegerType(),True),
                                                               StructField("bpr_tpn",IntegerType(),True)]),True),
                             StructField("rst_merged_values",
                                         ArrayType(StructType([StructField("rst_step_eff_dt",DateType(),True),
                                                               StructField("rst_step_end_dt",DateType(),True),
                                                               StructField("rst_step_type_num",IntegerType(),True),
                                                               StructField("rst_step_id",IntegerType(),True),
                                                               StructField("rst_underlying_step_id",IntegerType(),True),
                                                               StructField("rst_step_uplift",IntegerType(),True),
                                                               StructField("rst_step_volume",DoubleType(),True),
                                                               StructField("rst_actual_nws",DoubleType(),True),
                                                               StructField("rst_wthr_sens_flag",StringType(),True),
                                                               StructField("rst_continue_nws",StringType(),True)]),True),
                                     True)
                        ]
                       )
    rst_merged_rdd = (rst_sel_data
                         .withColumn("key",struct([col("ro_no"),col("bpr_tpn")]))
                         .withColumn("value",array([struct([
                                                           col("rst_step_eff_dt"),
                                                           col("rst_step_end_dt"),
                                                           col("rst_step_type_num"),
                                                           col("rst_step_id"),
                                                           col("rst_underlying_step_id"),
                                                           col("rst_step_uplift"),
                                                           col("rst_step_volume"),
                                                           col("rst_actual_nws"),
                                                           col("rst_wthr_sens_flag"),
                                                           col("rst_continue_nws")
                                                           ]
                                                          )
                                                 ])
                                    )
                         .select("key","value")
                         .rdd
                         .reduceByKey(lambda v1,v2 : v1 + v2)
                         #.toDF(["rst_key","rst_merged_values"])
                        )

    rst_merged_df = sqlContext.createDataFrame(rst_merged_rdd,rst_schema)
    
rst_merged_df.show(2)

+---------------+--------------------+
|        rst_key|   rst_merged_values|
+---------------+--------------------+
|[2920,81003974]|[[2017-10-16,2017...|
|[5992,50965158]|[[2017-10-16,2017...|
+---------------+--------------------+
only showing top 2 rows



In [32]:
join_cond2 = [nws_rrt_df.nws_key == rst_merged_df.rst_key]

nws_rrt_rst_df = nws_rrt_df.join(rst_merged_df,join_cond2,"leftouter").drop("rst_key")#.cache()

nws_rrt_rst_df.show(3)

+---------------+--------------------+-----------------+--------------------+
|        nws_key|   nws_merged_values|rrt_merged_values|   rst_merged_values|
+---------------+--------------------+-----------------+--------------------+
|[2139,50502413]|[[F13DA,W,Y,277.0...|             null|                null|
|[2518,54550994]|[[F51CA,A,N,588.8...|             null|                null|
|[2564,50965158]|[[F11MA,W,Y,18.77...|             null|[[2017-10-16,2017...|
+---------------+--------------------+-----------------+--------------------+
only showing top 3 rows



In [33]:

dst_sel_data = dst_data.select(col("RETAIL_OUTLET_NUMBER").alias("ro_no").cast(IntegerType()),
                               col("BASE_PRODUCT_NUMBER").alias("bpr_tpn").cast(IntegerType()),
                               col("START_DATE").alias("dst_start_date").cast(DateType()),
                               col("END_DATE").alias("dst_end_date").cast(DateType()),
                               col("STEP_ID").alias("dst_step_id").cast(IntegerType()),
                               col("UNDERLYING_STEP_ID").alias("dst_underlying_step_id").cast(IntegerType()),
                               col("STACK_INDEX").alias("dst_stack_index").cast(IntegerType())
                              )

dst_sel_data.show(3)

+-----+--------+--------------+------------+-----------+----------------------+---------------+
|ro_no| bpr_tpn|dst_start_date|dst_end_date|dst_step_id|dst_underlying_step_id|dst_stack_index|
+-----+--------+--------------+------------+-----------+----------------------+---------------+
| 2098|84128142|    2017-10-16|  2017-10-31|  129238922|                     0|              1|
| 6331|82639698|    2017-10-11|  2017-11-21|  119999696|                     0|              1|
| 2061|57092822|    2017-10-25|  2017-11-14|  115615265|                     0|              1|
+-----+--------+--------------+------------+-----------+----------------------+---------------+
only showing top 3 rows



In [34]:
if dst_sel_data.rdd.isEmpty():
    schema=StructType([StructField("ro_no", IntegerType(), True), StructField("bpr_tpn", IntegerType(), True), StructField("mergedInput", StringType(), True)])
    dst_merged_df = sqlContext.createDataFrame(sc.parallelize([(0,0,'0')]),schema=schema)
    dst_merged_rdd = dst_merged_df.rdd.map(lambda row : ((row.dst_ro_no,row.dst_bpr_tpn),row.mergedInput))
    dst_merged_df = dst_merged_rdd.toDF(["dst_key","dst_merged_values"])
else:
    
    if dst_sel_data.rdd.getNumPartitions() < 400:
            dst_sel_data  = dst_sel_data.repartition(400)
    
    dst_schema = StructType([StructField("dst_key",StructType([StructField("ro_no",IntegerType(),True),
                                                               StructField("bpr_tpn",IntegerType(),True)]),True),
                             StructField("dst_merged_values",
                                         ArrayType(StructType([StructField("dst_start_date",DateType(),True),
                                                               StructField("dst_end_date",DateType(),True),
                                                               StructField("dst_step_id",IntegerType(),True),
                                                               StructField("dst_underlying_step_id",IntegerType(),True),
                                                               StructField("dst_stack_index",IntegerType(),True)]),True),
                                     True)
                        ]
                       )
    dst_merged_rdd = (dst_sel_data
                         .withColumn("key",struct([col("ro_no"),
                                                   col("bpr_tpn")]))
                         .withColumn("value",array([struct([col("dst_start_date"),
                                                             col("dst_end_date"),
                                                             col("dst_step_id"),
                                                             col("dst_underlying_step_id"),
                                                             col("dst_stack_index")]
                                                          )
                                                 ])
                                    )
                         .select("key","value")
                         .rdd
                         .reduceByKey(lambda v1,v2 : v1 + v2)
                         #.toDF(["dst_key","dst_merged_values"])
                        )
    dst_merged_df = sqlContext.createDataFrame(dst_merged_rdd,dst_schema)
dst_merged_df.show(3)

+---------------+--------------------+
|        dst_key|   dst_merged_values|
+---------------+--------------------+
|[5570,64438600]|[[2017-10-23,2017...|
|[3236,51820386]|[[2017-11-15,2017...|
|[2202,81576272]|[[2017-10-04,2017...|
+---------------+--------------------+
only showing top 3 rows



In [35]:
join_cond2 = [nws_rrt_rst_df.nws_key == dst_merged_df.dst_key]

nws_rrt_rst_dst_df = nws_rrt_rst_df.join(dst_merged_df,join_cond2,"leftouter").drop("dst_key")#.cache()

nws_rrt_rst_dst_df.show(3)

+---------------+--------------------+-----------------+--------------------+--------------------+
|        nws_key|   nws_merged_values|rrt_merged_values|   rst_merged_values|   dst_merged_values|
+---------------+--------------------+-----------------+--------------------+--------------------+
|[2139,50502413]|[[F13DA,W,Y,277.0...|             null|                null|                null|
|[2518,54550994]|[[F51CA,A,N,588.8...|             null|                null|                null|
|[2564,50965158]|[[F11MA,W,Y,18.77...|             null|[[2017-10-16,2017...|[[2017-10-16,2017...|
+---------------+--------------------+-----------------+--------------------+--------------------+
only showing top 3 rows



In [36]:
dtp_sel_data = dtp_data.filter(dtp_data.year_week_number >= cal_yrwk_start).filter(dtp_data.year_week_number <= cal_yrwk_end)\
.select(col("RETAIL_OUTLET_NUMBER").alias("ro_no").cast(IntegerType()),
                               col("BASE_PRODUCT_NUMBER").alias("bpr_tpn").cast(IntegerType()),
                               col("YEAR_WEEK_NUMBER").alias("dtp_year_week").cast(StringType()),
                               col("PTP_1").alias("ptp_1").cast(DoubleType()),
                               col("PTP_2").alias("ptp_2").cast(DoubleType()),
                               col("PTP_3").alias("ptp_3").cast(DoubleType()),
                               col("PTP_4").alias("ptp_4").cast(DoubleType()),
                               col("PTP_5").alias("ptp_5").cast(DoubleType()),
                               col("PTP_6").alias("ptp_6").cast(DoubleType()),
                               col("PTP_7").alias("ptp_7").cast(DoubleType())
                               )#.cache()

dtp_sel_data.filter(dtp_sel_data.ro_no==2132).filter(dtp_sel_data.bpr_tpn==50934973).show(10)

+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+
|ro_no| bpr_tpn|dtp_year_week|ptp_1|ptp_2|ptp_3|ptp_4|ptp_5|ptp_6|ptp_7|
+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+
| 2132|50934973|       201733| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201734| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201741| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201745| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201742| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201737| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201739| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201743| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201738| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
| 2132|50934973|       201736| 15.6| 12.4| 11.9| 13.4| 17.0| 19.2| 10.5|
+-----+--------+-------------+-----+-----+-----+---

In [37]:
# Join DTP with the RST grouped at store product level to decide the range of dates for which the DTP needs to be picked
dtp_join_cond = [dtp_sel_data.ro_no == rst_sel_data_groupBy_3.ro_no
                ,dtp_sel_data.bpr_tpn == rst_sel_data_groupBy_3.bpr_tpn]

dtp_sel_data_2 = (dtp_sel_data.join(rst_sel_data_groupBy_3
                                    ,dtp_join_cond
                                    ,"left_outer")
                                    .drop(rst_sel_data_groupBy_3.ro_no)
                                    .drop(rst_sel_data_groupBy_3.bpr_tpn))
        
dtp_sel_data_2.show(3)

+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+-------------------+-----------------+------------------------+----------------------+
|ro_no| bpr_tpn|dtp_year_week|ptp_1|ptp_2|ptp_3|ptp_4|ptp_5|ptp_6|ptp_7|min_fcst_start_date|max_fcst_end_date|min_fcst_start_year_week|max_fcst_end_year_week|
+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+-------------------+-----------------+------------------------+----------------------+
| 2894|50502269|       201742| 13.9| 11.6| 12.1| 15.0| 18.4| 17.4| 11.6|         2017-10-16|       2018-01-15|                  201734|                201747|
| 2894|50502269|       201748| 13.9| 11.6| 12.1| 15.0| 18.4| 17.4| 11.6|         2017-10-16|       2018-01-15|                  201734|                201747|
| 2894|50502269|       201738| 13.9| 11.6| 12.1| 15.0| 18.4| 17.4| 11.6|         2017-10-16|       2018-01-15|                  201734|                201747|
+-----+--------+-------------+-----+-----+----

In [38]:
# Replace all null values of min_fcst_start_year_week and max_fcst_end_year_week with forecast start and end year week respectively

dtp_sel_data_3 = dtp_sel_data_2.na.fill({"min_fcst_start_year_week" : cal_fcst_start_year_week,
                                         "max_fcst_end_year_week" : cal_fcst_end_year_week})

dtp_sel_data_3.show(3)

+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+-------------------+-----------------+------------------------+----------------------+
|ro_no| bpr_tpn|dtp_year_week|ptp_1|ptp_2|ptp_3|ptp_4|ptp_5|ptp_6|ptp_7|min_fcst_start_date|max_fcst_end_date|min_fcst_start_year_week|max_fcst_end_year_week|
+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+-------------------+-----------------+------------------------+----------------------+
| 2894|50502269|       201745| 13.9| 11.6| 12.1| 15.0| 18.4| 17.4| 11.6|         2017-10-16|       2018-01-15|                  201734|                201747|
| 2894|50502269|       201734| 13.9| 11.6| 12.1| 15.0| 18.4| 17.4| 11.6|         2017-10-16|       2018-01-15|                  201734|                201747|
| 2894|50502269|       201740| 13.9| 11.6| 12.1| 15.0| 18.4| 17.4| 11.6|         2017-10-16|       2018-01-15|                  201734|                201747|
+-----+--------+-------------+-----+-----+----

In [39]:
dtp_sel_data = (dtp_sel_data_3.filter(dtp_sel_data_3.dtp_year_week >= dtp_sel_data_3.min_fcst_start_year_week )
                              .filter(dtp_sel_data_3.dtp_year_week <= dtp_sel_data_3.max_fcst_end_year_week))
    
#dtp_sel_data.show()

In [40]:
if dtp_sel_data.rdd.isEmpty():
    schema=StructType([StructField("dtp_ro_no", IntegerType(), True), StructField("dtp_bpr_tpn", IntegerType(), True), StructField("mergedInput", StringType(), True)])
    dtp_merged_df = sqlContext.createDataFrame(sc.parallelize([(0,0,'0')]),schema=schema)
    dtp_merged_rdd = dtp_merged_df.rdd.map(lambda row : ((row.dtp_ro_no,row.dtp_bpr_tpn),row.mergedInput))
    dtp_merged_df = dtp_merged_rdd.toDF(["dtp_key","dtp_merged_values"])
else:  
    
    if dtp_sel_data.rdd.getNumPartitions() < 400:
        dtp_sel_data  = dtp_sel_data.repartition(400)
  
    dtp_schema = StructType([StructField("dtp_key",StructType([StructField("ro_no",IntegerType(),True),
                                                               StructField("bpr_tpn",IntegerType(),True)]),True),
                             StructField("dtp_merged_values",
                                         ArrayType(StructType([StructField("dtp_year_week",StringType(),True),
                                                               StructField("ptp_1",DoubleType(),True),
                                                               StructField("ptp_2",DoubleType(),True),
                                                               StructField("ptp_3",DoubleType(),True),
                                                               StructField("ptp_4",DoubleType(),True),
                                                               StructField("ptp_5",DoubleType(),True),
                                                               StructField("ptp_6",DoubleType(),True),
                                                               StructField("ptp_7",DoubleType(),True)]),True),
                                     True)
                        ]
                       )
    dtp_merged_rdd = (dtp_sel_data
                         .withColumn("key",struct([col("ro_no"),
                                                   col("bpr_tpn")]))
                         .withColumn("value",array([struct([col("dtp_year_week"),
                                                             col("ptp_1"),
                                                             col("ptp_2"),
                                                             col("ptp_3"),
                                                             col("ptp_4"),
                                                             col("ptp_5"),
                                                             col("ptp_6"),
                                                             col("ptp_7")])
                                                 ])
                                    )
                         .select("key","value")
                         .rdd
                         .reduceByKey(lambda v1,v2 : v1 + v2)
                         #.toDF(["dtp_key","dtp_merged_values"])
                        )
    
    dtp_merged_df = sqlContext.createDataFrame(dtp_merged_rdd,dtp_schema)

dtp_merged_df.show(3)

+---------------+--------------------+
|        dtp_key|   dtp_merged_values|
+---------------+--------------------+
|[5099,51252015]|[[201735,14.4,14....|
|[6058,52466256]|[[201744,17.0,14....|
|[5992,50965158]|[[201737,14.3,12....|
+---------------+--------------------+
only showing top 3 rows



In [41]:
ptg_sel_data = ptg_data.filter(ptg_data.year_week_number >= cal_yrwk_start).filter(ptg_data.year_week_number <= cal_yrwk_end)\
.select(col("RETAIL_OUTLET_NUMBER").alias("ro_no").cast(IntegerType()),
                               col("BASE_PRODUCT_NUMBER").alias("bpr_tpn").cast(IntegerType()),
                               col("YEAR_WEEK_NUMBER").alias("ptg_year_week").cast(StringType()),
                               col("PTG_1").alias("ptg_1").cast(DoubleType()),
                               col("PTG_2").alias("ptg_2").cast(DoubleType()),
                               col("PTG_3").alias("ptg_3").cast(DoubleType()),
                               col("PTG_4").alias("ptg_4").cast(DoubleType()),
                               col("PTG_5").alias("ptg_5").cast(DoubleType()),
                               col("PTG_6").alias("ptg_6").cast(DoubleType()),
                               col("PTG_7").alias("ptg_7").cast(DoubleType())
                               )


ptg_sel_data =  (ptg_sel_data.filter(col('ptg_1').isNotNull()) 
                            .filter(col('ptg_2').isNotNull()) 
                            .filter(col('ptg_3').isNotNull())
                            .filter(col('ptg_4').isNotNull()) 
                            .filter(col('ptg_5').isNotNull()) 
                            .filter(col('ptg_6').isNotNull()) 
                            .filter(col('ptg_7').isNotNull())
                           # .cache()
                )
ptg_sel_data.show(3)

+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+
|ro_no| bpr_tpn|ptg_year_week|ptg_1|ptg_2|ptg_3|ptg_4|ptg_5|ptg_6|ptg_7|
+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+
| 5626|78539177|       201736| 18.5| 14.8| 16.7| 14.3| 13.2| 13.8|  9.8|
| 2532|65729767|       201736| 13.9| 12.9| 11.8| 14.8| 17.1| 19.5| 10.8|
| 3411|52466256|       201735| 14.7| 12.2| 11.4| 14.4| 18.3| 18.0| 12.0|
+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+
only showing top 3 rows



In [42]:
# Join PTG with the RST grouped at store product level to decide the range of dates for which the PTG needs to be picked
ptg_join_cond = [ptg_sel_data.ro_no == rst_sel_data_groupBy_3.ro_no
                ,ptg_sel_data.bpr_tpn == rst_sel_data_groupBy_3.bpr_tpn]

ptg_sel_data_2 = (ptg_sel_data.join(rst_sel_data_groupBy_3
                                    ,ptg_join_cond
                                    ,"left_outer")
                                    .drop(rst_sel_data_groupBy_3.ro_no)
                                    .drop(rst_sel_data_groupBy_3.bpr_tpn))

ptg_sel_data_2.show(3)

+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+-------------------+-----------------+------------------------+----------------------+
|ro_no| bpr_tpn|ptg_year_week|ptg_1|ptg_2|ptg_3|ptg_4|ptg_5|ptg_6|ptg_7|min_fcst_start_date|max_fcst_end_date|min_fcst_start_year_week|max_fcst_end_year_week|
+-----+--------+-------------+-----+-----+-----+-----+-----+-----+-----+-------------------+-----------------+------------------------+----------------------+
| 2894|50502269|       201735| 13.9| 11.6| 12.1| 15.3| 18.8| 17.9| 12.1|         2017-10-16|       2018-01-15|                  201734|                201747|
| 2894|50502269|       201736| 14.3| 12.0| 12.1| 15.0| 18.4| 17.4| 11.6|         2017-10-16|       2018-01-15|                  201734|                201747|
| 2388|54550994|       201736| 14.2| 13.5| 11.7| 13.9| 17.7| 15.5| 14.1|         2017-10-13|       2018-01-15|                  201733|                201747|
+-----+--------+-------------+-----+-----+----

In [44]:
# Replace all null values of min_fcst_start_year_week and max_fcst_end_year_week with forecast start and end year week respectively

ptg_sel_data_3 = ptg_sel_data_2.na.fill({"min_fcst_start_year_week" : cal_fcst_start_year_week,
                                         "max_fcst_end_year_week" : cal_fcst_end_year_week})



In [46]:
ptg_sel_data = (ptg_sel_data_3.filter(ptg_sel_data_3.ptg_year_week >= ptg_sel_data_3.min_fcst_start_year_week )
                              .filter(ptg_sel_data_3.ptg_year_week <= ptg_sel_data_3.max_fcst_end_year_week))
    

In [47]:
if ptg_sel_data.rdd.isEmpty():
    schema=StructType([StructField("ro_no", IntegerType(), True), StructField("bpr_tpn", IntegerType(), True), StructField("mergedInput", StringType(), True)])
    ptg_merged_df = sqlContext.createDataFrame(sc.parallelize([(0,0,'0')]),schema=schema)
    ptg_merged_rdd = ptg_merged_df.rdd.map(lambda row : ((row.ptg_ro_no,row.ptg_bpr_tpn),row.mergedInput))
    ptg_merged_df = ptg_merged_rdd.toDF(["ptg_key","ptg_merged_values"])
else:
    
    if ptg_sel_data.rdd.getNumPartitions() < 400:
        ptg_sel_data  = ptg_sel_data.repartition(400)
    
    ptg_schema = StructType([StructField("ptg_key",StructType([StructField("ro_no",IntegerType(),True),
                                                               StructField("bpr_tpn",IntegerType(),True)]),True),
                             StructField("ptg_merged_values",
                                         ArrayType(StructType([StructField("ptg_year_week",StringType(),True),
                                                               StructField("ptg_1",DoubleType(),True),
                                                               StructField("ptg_2",DoubleType(),True),
                                                               StructField("ptg_3",DoubleType(),True),
                                                               StructField("ptg_4",DoubleType(),True),
                                                               StructField("ptg_5",DoubleType(),True),
                                                               StructField("ptg_6",DoubleType(),True),
                                                               StructField("ptg_7",DoubleType(),True)]),True),
                                     True)
                        ]
                       )
    ptg_merged_rdd = (ptg_sel_data
                         .withColumn("key",struct([col("ro_no"),
                                                   col("bpr_tpn")
                                                  ]))
                         .withColumn("value",array([struct([col("ptg_year_week"),
                                                           col("ptg_1"),
                                                           col("ptg_2"),
                                                           col("ptg_3"),
                                                           col("ptg_4"),
                                                           col("ptg_5"),
                                                           col("ptg_6"),
                                                           col("ptg_7")
                                                           ])
                                                 ])
                                    )
                         .select("key","value")
                         .rdd
                         .reduceByKey(lambda v1,v2 : v1 + v2)
                         #.toDF(["ptg_key","ptg_merged_values"])
                    )
    ptg_merged_df = sqlContext.createDataFrame(ptg_merged_rdd,ptg_schema)
    
ptg_merged_df.show(3)

+---------------+--------------------+
|        ptg_key|   ptg_merged_values|
+---------------+--------------------+
|[2401,68086237]|[[201736,15.2,13....|
|[2920,81003974]|[[201736,16.5,14....|
|[5992,50965158]|[[201735,14.3,12....|
+---------------+--------------------+
only showing top 3 rows



In [48]:
join_cond3 = [nws_rrt_rst_dst_df.nws_key == ptg_merged_df.ptg_key]

nws_rrt_rst_dst_ptg_df = nws_rrt_rst_dst_df.join(ptg_merged_df,join_cond3,"leftouter").drop("ptg_key")#.cache()

#nws_rrt_rst_dst_ptg_df.show()

In [49]:
join_cond4 = [nws_rrt_rst_dst_ptg_df.nws_key == dtp_merged_df.dtp_key]

nws_rrt_rst_dst_ptg_dtp_df = nws_rrt_rst_dst_ptg_df.join(dtp_merged_df,join_cond4,"leftouter").drop("dtp_key")#.cache()

nws_rrt_rst_dst_ptg_dtp_df.show(3)

+---------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|        nws_key|   nws_merged_values|rrt_merged_values|   rst_merged_values|   dst_merged_values|   ptg_merged_values|   dtp_merged_values|
+---------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|[2139,50502413]|[[F13DA,W,Y,277.0...|             null|                null|                null|[[201735,14.1,12....|[[201736,14.1,12....|
|[2518,54550994]|[[F51CA,A,N,588.8...|             null|                null|                null|                null|[[201746,15.1,13....|
|[2564,50965158]|[[F11MA,W,Y,18.77...|             null|[[2017-10-16,2017...|[[2017-10-16,2017...|[[201736,14.7,12....|[[201734,14.1,12....|
+---------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
only showing 

In [50]:

swf_sel_data = swf_data.select(col("AGGR_GRP_ID").alias("swf_aggr_grp_id").cast(StringType()),
                               col("PRODUCT_SUB_GROUP_CODE").alias("swf_sg_cd").cast(StringType()),
                               col("WTHR_TESCO_WEEK").alias("swf_wthr_tesco_week").cast(StringType()),
                               col("TESCO_DAY").alias("swf_tesco_day").cast(StringType()),
                               col("WTHR_FACTOR").alias("swf_wthr_factor").cast(DoubleType())
                               )
swf_sel_data.show(3)

+---------------+---------+-------------------+-------------+------------------+
|swf_aggr_grp_id|swf_sg_cd|swf_wthr_tesco_week|swf_tesco_day|   swf_wthr_factor|
+---------------+---------+-------------------+-------------+------------------+
|       WTHEIXMN|    G14GD|                 36|            6|0.9998823404312134|
|       WTHWAXM |    G41LD|                 38|            1| 1.116187572479248|
|       WTHMINM |    F11BE|                 37|            5|0.8947426080703735|
+---------------+---------+-------------------+-------------+------------------+
only showing top 3 rows



In [51]:
join_cond7 = [nws_sel_data.aggr_grp_id == swf_sel_data.swf_aggr_grp_id, nws_sel_data.sg_cd == swf_sel_data.swf_sg_cd]
swf_sel_data = nws_sel_data.join(swf_sel_data,join_cond7).select('ro_no','bpr_tpn','swf_wthr_tesco_week','swf_tesco_day','swf_wthr_factor')
#swf_sel_data.show()

In [52]:
if swf_sel_data.rdd.isEmpty():
    schema=StructType([StructField("ro_no", IntegerType(), True), StructField("bpr_tpn", IntegerType(), True), StructField("mergedInput", StringType(), True)])
    swf_merged_df = sqlContext.createDataFrame(sc.parallelize([(0,0,'0')]),schema=schema)
    swf_merged_rdd = swf_merged_df.rdd.map(lambda row : ((row.swf_ro_no,row.swf_bpr_tpn),row.mergedInput))
    swf_merged_df = swf_merged_rdd.toDF(["swf_key","swf_merged_values"])
else:    
    if swf_sel_data.rdd.getNumPartitions() < 400:
        swf_sel_data  = swf_sel_data.repartition(400)
    
    swf_schema = StructType([StructField("swf_key",StructType([StructField("ro_no",IntegerType(),True),
                                                               StructField("bpr_tpn",IntegerType(),True)]),True),
                             StructField("swf_merged_values",
                                         ArrayType(StructType([StructField("swf_wthr_tesco_week",StringType(),True),
                                                               StructField("swf_tesco_day",StringType(),True),
                                                               StructField("swf_wthr_factor",DoubleType(),True)]),True),
                                     True)
                        ]
                       )

    swf_merged_rdd = (swf_sel_data
                         .withColumn("key",struct([col("ro_no"),
                                                   col("bpr_tpn")
                                                  ]))
                         .withColumn("value",array([struct([col("swf_wthr_tesco_week"),
                                                           col("swf_tesco_day"),
                                                           col("swf_wthr_factor")
                                                           ])
                                                 ])
                                    )
                         .select("key","value")
                         .rdd
                         .reduceByKey(lambda v1,v2 : v1 + v2)
                         #.toDF(["swf_key","swf_merged_values"])
                    )
    swf_merged_df = sqlContext.createDataFrame(swf_merged_rdd,swf_schema)

swf_merged_df.show(3)

+---------------+--------------------+
|        swf_key|   swf_merged_values|
+---------------+--------------------+
|[5099,51252015]|[[36,3,1.0], [35,...|
|[2920,81003974]|[[36,6,0.95935678...|
|[2129,68086237]|[[35,3,1.00130093...|
+---------------+--------------------+
only showing top 3 rows



In [53]:
join_cond5 = [nws_rrt_rst_dst_ptg_dtp_df.nws_key == swf_merged_df.swf_key]

nws_rrt_rst_dst_ptg_dtp_swf_df = nws_rrt_rst_dst_ptg_dtp_df.join(swf_merged_df,join_cond5,"leftouter").drop("swf_key")

nws_rrt_rst_dst_ptg_dtp_swf_df.show(3)

+---------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        nws_key|   nws_merged_values|rrt_merged_values|   rst_merged_values|   dst_merged_values|   ptg_merged_values|   dtp_merged_values|   swf_merged_values|
+---------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[2139,50502413]|[[F13DA,W,Y,277.0...|             null|                null|                null|[[201735,14.1,12....|[[201736,14.1,12....|[[43,3,0.77321857...|
|[2518,54550994]|[[F51CA,A,N,588.8...|             null|                null|                null|                null|[[201746,15.1,13....|[[42,3,1.0], [42,...|
|[2564,50965158]|[[F11MA,W,Y,18.77...|             null|[[2017-10-16,2017...|[[2017-10-16,2017...|[[201736,14.7,12....|[[201734,14.1,12....|[[40,3,0.83278703...|
+---------------+-----------

In [54]:
wef_sel_data = wef_data.select(col("AGGR_GRP_ID").alias("wef_aggr_grp_id").cast(StringType()),
                               col("PRODUCT_SUB_GROUP_CODE").alias("wef_sg_cd").cast(StringType()),
                               col("SALES_DATE").alias("wef_sales_date").cast(StringType()),
                               col("WTHR_FACTOR").alias("wef_wthr_factor").cast(DoubleType())
                               )
#wef_sel_data.show()

In [55]:
join_cond8 = [nws_sel_data.aggr_grp_id == wef_sel_data.wef_aggr_grp_id, nws_sel_data.sg_cd == wef_sel_data.wef_sg_cd]
wef_sel_data = nws_sel_data.join(wef_sel_data,join_cond8).select('ro_no','bpr_tpn','wef_sales_date','wef_wthr_factor')
wef_sel_data.show(3)

+-----+--------+--------------+------------------+
|ro_no| bpr_tpn|wef_sales_date|   wef_wthr_factor|
+-----+--------+--------------+------------------+
| 2547|50935010|    2017-09-20|1.0216832160949707|
| 2547|50935010|    2017-09-13|               1.0|
| 2547|50935010|    2017-09-19|1.0241930484771729|
+-----+--------+--------------+------------------+
only showing top 3 rows



In [56]:
if wef_sel_data.rdd.isEmpty():
    schema=StructType([StructField("ro_no", IntegerType(), True), StructField("bpr_tpn", IntegerType(), True), StructField("mergedInput", StringType(), True)])
    wef_merged_df = sqlContext.createDataFrame(sc.parallelize([(0,0,'0')]),schema=schema)
    wef_merged_rdd = wef_merged_df.rdd.map(lambda row : ((row.wef_ro_no,row.wef_bpr_tpn),row.mergedInput))
    wef_merged_df = wef_merged_rdd.toDF(["wef_key","wef_merged_values"])
else:
    if wef_sel_data.rdd.getNumPartitions() < 400:
        wef_sel_data  = wef_sel_data.repartition(400)
    
    wef_schema = StructType([StructField("wef_key",StructType([StructField("ro_no",IntegerType(),True),
                                                               StructField("bpr_tpn",IntegerType(),True)]),True),
                             StructField("wef_merged_values",
                                         ArrayType(StructType([StructField("wef_sales_date",StringType(),True),
                                                               StructField("wef_wthr_factor",DoubleType(),True)]),True),
                                     True)
                        ]
                       )
    
    wef_merged_rdd = (wef_sel_data
                         .withColumn("key",struct([col("ro_no"),
                                                   col("bpr_tpn")
                                                  ]))
                         .withColumn("value",array([struct([col("wef_sales_date"),
                                                           col("wef_wthr_factor")
                                                           ])
                                                 ])
                                    )
                         .select("key","value")
                         .rdd
                         .reduceByKey(lambda v1,v2 : v1 + v2)
                         #.toDF(["wef_key","wef_merged_values"])
                    )
    wef_merged_df = sqlContext.createDataFrame(wef_merged_rdd,wef_schema)

wef_merged_df.show(3)

+---------------+--------------------+
|        wef_key|   wef_merged_values|
+---------------+--------------------+
|[2920,81003974]|[[2017-10-10,1.00...|
|[6057,57477477]|[[2017-11-10,0.92...|
|[5157,57755937]|[[2017-10-14,1.07...|
+---------------+--------------------+
only showing top 3 rows



In [57]:
join_cond6 = [nws_rrt_rst_dst_ptg_dtp_swf_df.nws_key == wef_merged_df.wef_key]
nws_rrt_rst_dst_ptg_dtp_swf_wef_df = nws_rrt_rst_dst_ptg_dtp_swf_df.join(wef_merged_df,join_cond6,"leftouter").drop("wef_key")#.persist(StorageLevel.MEMORY_AND_DISK_SER)
nws_rrt_rst_dst_ptg_dtp_swf_wef_df.show(3)

+---------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        nws_key|   nws_merged_values|rrt_merged_values|   rst_merged_values|   dst_merged_values|   ptg_merged_values|   dtp_merged_values|   swf_merged_values|   wef_merged_values|
+---------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[2139,50502413]|[[F13DA,W,Y,277.0...|             null|                null|                null|[[201735,14.1,12....|[[201736,14.1,12....|[[34,1,0.89153015...|[[2017-10-21,0.95...|
|[2518,54550994]|[[F51CA,A,N,588.8...|             null|                null|                null|                null|[[201746,15.1,13....|[[42,3,1.0], [42,...|                null|
|[2564,50965158]|[[F11MA,W,Y,18.77...|             null|[[2017-10-16,2017...|[[2017-1

In [58]:
#nws_rrt_rst_dst_ptg_dtp_swf_wef_df.write.format('orc').save(path='hdfs://pphdp/insight_labs/rdf/temp/gct8/intermediate_output_allStores_200by4_'+sub_group+'_'+current_date,mode='overwrite')
nws_rrt_rst_dst_ptg_dtp_swf_wef_df.write.format('orc').save(path='hdfs://pphdp/insight_labs/rdf/output/cr_forecast/intermediate_output_allStores_'+sub_group+'_'+current_date+'_'+'merged_version',mode='overwrite')

In [59]:
print datetime.now()

2017-10-16 06:25:01.528909


In [60]:
from pyspark.sql.types import DateType, DoubleType, IntegerType, StringType, StructField, StructType, DecimalType

def forecast_data_schema():
    return StructType([
        StructField("Retail_Outlet_Number", IntegerType(), True),
        StructField("Base_Product_Number", IntegerType(), True),
        StructField("Run_Date", StringType(), True),
        StructField("Sub_Group_Code", StringType(), True),
        StructField("Forecast_Date", StringType(), True),
        StructField("Forecast_NWS", DoubleType(), True),
        StructField("NWS_Stat", StringType(), True),
        StructField("Daily_PTP", DoubleType(), True),
        StructField("Weather_Factor", DoubleType(), True),
        StructField("Expected_Daily_Sales", DoubleType(), True)
    ])

In [61]:
### These functions are meant for converting the numpy arrays to pandas dataframes for the ease of accessing the same
def getBaseAttributes(nws_list):
    nws_pd = pd.DataFrame(data=nws_list,
                          columns=['sg_cd','nws_stat','step_ind','final_nws','saved_final_nws','aggr_grp_id','stkd_prod_stdt','stkd_prod_endt']
                         )
    return nws_pd
    
def getResetAttributes(rrt_list):
    rrt_pd = pd.DataFrame(data=rrt_list,
                          columns=['rrt_step_eff_dt','rrt_step_type_num','rrt_step_uplift','rrt_step_volume']
                         )
    rr,rc = rrt_pd.shape
    rrt_exists = True if rr > 0 else False
    return rr, rc, rrt_pd, rrt_exists

def getStepAttributes(rst_list):
    rst_pd = pd.DataFrame(data=rst_list,
                          columns=['rst_step_eff_dt','rst_step_end_dt','rst_step_type_num','rst_step_id','rst_underlying_step_id','rst_step_uplift','rst_step_volume','rst_actual_nws','rst_wthr_sens_flag','rst_continue_nws']
                         )
    sr,sc = rst_pd.shape
    rst_exists = True if sr > 0 else False
    return sr, sc, rst_pd, rst_exists

def getDailyAttributes(dst_list):
    dst_pd = pd.DataFrame(data=dst_list,
                          columns=['dst_start_date','dst_end_date','dst_step_id','dst_underlying_step_id','dst_stack_index']
                         )
    dr,dc = dst_pd.shape
    dst_exists = True if dr > 0 else False
    return dr, dc, dst_pd, dst_exists

def getEventAttributes(ptg_list):
    ptg_pd = pd.DataFrame(data=ptg_list,
                          columns=['ptg_year_week','ptg_1','ptg_2','ptg_3','ptg_4','ptg_5','ptg_6','ptg_7']
                         )
    gr,gc = ptg_pd.shape
    ptg_exists = True if gr > 0 else False
    return gr, gc, ptg_pd, ptg_exists

def getTradingAttributes(dtp_list):
    dtp_pd = pd.DataFrame(data=dtp_list,
                          columns=['ptp_year_week','ptp_1','ptp_2','ptp_3','ptp_4','ptp_5','ptp_6','ptp_7']
                         )
    pr,pc = dtp_pd.shape
    dtp_exists = True if pr > 0 else False
    return pr, pc, dtp_pd, dtp_exists

def getSeasonalAttributes(swf_list):
    swf_pd = pd.DataFrame(data=swf_list,
                          columns=['swf_wthr_tesco_week','swf_tesco_day','swf_wthr_factor']
                         )
    fr,fc = swf_pd.shape
    swf_exists = True if fr > 0 else False
    return fr, fc, swf_pd, swf_exists

def getWeatherAttributes(wef_list):
    wef_pd = pd.DataFrame(data=wef_list,
                          columns=['wef_sales_date','wef_wthr_factor']
                         )
    wr,wc = wef_pd.shape
    wef_exists = True if wr > 0 else False
    return wr, wc, wef_pd, wef_exists

In [62]:
### These functions return the PTG and PTP based on year week that is passed

def getDaysPtg(gr,ptg_pd,year_week):
    for p in np.where(ptg_pd.ptg_year_week == year_week[0])[0]:
        days_ptp = eval("ptg_pd.ptg_"+str(year_week[2])+"[p]")
        return (days_ptp,True)
    return (0,False)
        
def getDaysPtp(pr,dtp_pd,year_week):
    for q in np.where(dtp_pd.ptp_year_week == year_week[0])[0]:
        days_ptp = eval("dtp_pd.ptp_"+str(year_week[2])+"[q]")
        return days_ptp
    return 14.3

# This function returns the Seasonal Weather Factor for a day
def getDaysSeasonality(fr,swf_pd,year_week):
    for r in np.where((year_week[1] == swf_pd.swf_wthr_tesco_week) & (year_week[2] == swf_pd.swf_tesco_day))[0]:
        return swf_pd.swf_wthr_factor[r]
    return 1.0

# This function returns the Real Weather Factor for a day
def getRealWF(day_no,wef_pd,wr,swf_pd,fr,year_week):
    iso_date = day_no.strftime("%Y-%m-%d")
    for d in np.where(wef_pd.wef_sales_date == iso_date)[0]:
        return wef_pd.wef_wthr_factor[d]
    return getDaysSeasonality(fr,swf_pd,year_week)

In [63]:
# This function is meant to return the actual NWS from a back to back promotion
def getB2BPromo(rst_pd,sr,date):
    for b in range (sr):
        if date == rst_pd.rst_step_end_dt[b] \
        and rst_pd.rst_step_type_num[b] in [201,202,203,204]:
            if pd.notnull(rst_pd.rst_actual_nws[b]):
                return (rst_pd.rst_actual_nws[b],True)
    
    return (0,False)

In [64]:
# This function return the range of seven days from the day that is passed to it.
def getFutureDays(step_eff_dt,date_limit):
    step_date_plus_6 = step_eff_dt + timedelta(days=6)
    futureDays = []
    for key in sorted(date_limit.keys()):
        if key >= step_eff_dt and key <= step_date_plus_6:
            futureDays.append(date_limit[key])
    return futureDays

In [65]:
# This function calculates the Seasonal Weather Factor for the range of dates that is passed.
def getCalcSeasonalWF(range_of_dates,ptg_pd,ptg_exists,gr,gc,dtp_pd,pr,pc,dtp_exists,swf_pd,swf_exists,fr,fc):
    ptp = []
    swf = []
    ptg_found = False
    sum_of_ptps = 0.0
    sum_of_ptps_swf = 0.0
    
    for c in range (len(range_of_dates)):
        days_ptp = 0.142
        if ptg_exists:
            # Use PTG, if available
            (days_ptp,ptg_found) = getDaysPtg(gr,ptg_pd,range_of_dates[c])
        
        if not ptg_found:
            if dtp_exists:
                # If there exists no PTG, use PTP
                days_ptp = getDaysPtp(pr,dtp_pd,range_of_dates[c])
        ptp.append(days_ptp)
        
        days_swf = getDaysSeasonality(fr,swf_pd,range_of_dates[c])
        swf.append(days_swf)
            
    swf_into_ptp = [s * p for s,p in zip(ptp,swf)]
    
    sum_of_ptps_swf = reduce(lambda x,y : x+y ,swf_into_ptp)
    sum_of_ptps = reduce(lambda x,y : x+y ,ptp)
    
    if sum_of_ptps > 0:
        return sum_of_ptps_swf/sum_of_ptps
    else:
        return 1.0

In [66]:
# This function calculates the Real Weather Factor for the range of dates that is passed.
def getCalcRealWF(step_eff_dt,date_limit,ptg_pd,ptg_exists,gr,gc,dtp_pd,dtp_exists,pr,pc,
                  wef_pd,wef_exists,wr,wc,swf_pd,swf_exists,fr,fc):
    ptp = []
    wef = []
    ptg_found = False
    sum_of_ptps = 0.0
    sum_of_ptps_wef = 0.0
    
    step_date_plus_6 = step_eff_dt + timedelta(days=6)
    range_of_dates = []
    iso_dates = []
    
    for key in sorted(date_limit.keys()):
        if key >= step_eff_dt and key <= step_date_plus_6:
            iso_dates.append(key)
            range_of_dates.append(date_limit[key])
                    
    for c in range (len(range_of_dates)):
        days_ptp = 0.143
        if ptg_exists:
            # Use PTG, if available
            (days_ptp,ptg_found) = getDaysPtg(gr,ptg_pd,range_of_dates[c])
        
        if not ptg_found:
            if dtp_exists:
                # If there exists no PTG, use PTP
                days_ptp = getDaysPtp(pr,dtp_pd,range_of_dates[c])
        ptp.append(days_ptp)
        days_wef = getRealWF(iso_dates[c],wef_pd,wr,swf_pd,fr,range_of_dates[c])
        wef.append(days_wef)
            
    wef_into_ptp = [s * p for s,p in zip(ptp,wef)]
    
    sum_of_ptps_wef = reduce(lambda x,y : x+y ,wef_into_ptp)
    sum_of_ptps = reduce(lambda x,y : x+y ,ptp)
    
    if sum_of_ptps > 0:
        return sum_of_ptps_wef/sum_of_ptps
    else:
        return 1.0

In [67]:
# This function accepts all the attributes that influence the forecast for the store/product as one record 
# and returns the forecast for 'n' number of days
def getForecast(row):
    dst_list = np.array(row.dst_merged_values)    
    ptg_exists = False
    rrt_exists = False
    rst_exists = False
    dst_exists = False
    dtp_exists = False
    swf_exists = False
    wef_exists = False
    
    # Get the broadcated calendar table details
    date_range = cal_bc_var1.value
    swf_range = cal_bc_var2.value
    current_date = (datetime.now()+ timedelta(days=-1)).strftime("%Y-%m-%d")
    #current_date = '2017-08-16'
    
    # Lets convert the infleuncers into pd dataframe for further processing
    nws_pd = getBaseAttributes(row.nws_merged_values)
    
    (rr,rc,rrt_pd,rrt_exists) = getResetAttributes(row.rrt_merged_values if row.rrt_merged_values is not None else [])
    (sr,sc,rst_pd,rst_exists) = getStepAttributes(row.rst_merged_values if row.rst_merged_values is not None else [])
    (pr,pc,dtp_pd,dtp_exists) = getTradingAttributes(row.dtp_merged_values if row.dtp_merged_values is not None else [])
    (gr,gc,ptg_pd,ptg_exists) = getEventAttributes(row.ptg_merged_values if row.ptg_merged_values is not None else [])
    (fr,fc,swf_pd,swf_exists) = getSeasonalAttributes(row.swf_merged_values if row.swf_merged_values is not None else [])
    (wr,wc,wef_pd,wef_exists) = getWeatherAttributes(row.wef_merged_values if row.wef_merged_values is not None else [])
    
    if row.dst_merged_values is not None:
        dr,dc = dst_list.shape
        dst_exists = True      
        
    ###############################################################################
    # Start of forecast algorithm - For every day in the forecast window
    # 1. Apply volume/percentage reset to the base NWS
    # 2. If there is any step underlying the reset, reset it's volume and stepped nws accordingly
    # 3. Apply the effect of each step in the ascending order of their stack index 
    
    forecast_data = []
    forecast_data.extend([row.nws_key.ro_no,row.nws_key.bpr_tpn])
    
    # Get the appropriate NWS to start with : Use saved if available, else final NWS
    if pd.notnull(nws_pd.saved_final_nws[0]):
        base_nws = nws_pd.saved_final_nws[0]
        base_nws_reset = nws_pd.saved_final_nws[0]
    else:
        base_nws = nws_pd.final_nws[0]
        base_nws_reset = nws_pd.final_nws[0]
    
    # In case stocked start or end date is null, set it to maximum value
    maximum_date = datetime.strptime('99991231', "%Y%m%d").date()
    if pd.isnull(nws_pd.stkd_prod_stdt[0]):
        nws_pd.stkd_prod_stdt[0] = maximum_date
    
    if pd.isnull(nws_pd.stkd_prod_endt[0]):
        nws_pd.stkd_prod_endt[0] = maximum_date
    
    # For every day in the forecast window, loop through RRT, RST, DST to arrive at forecast NWS
    for day in sorted(date_range.keys()):
        forecast_date = day
        if forecast_date >= nws_pd.stkd_prod_stdt[0] \
        and forecast_date <= nws_pd.stkd_prod_endt[0]: 
            vol_reset = False
            per_reset = False
            ptg_found = False
            ptp_found = False
            b2b_promo_found = False
            base_nws = base_nws_reset
            total_reset_factor = 1
            reset_factor = 1
            
            #See if there are any resets for the forecast day. If so, reset the base accordingly
            if rrt_exists:
                for j in np.where(rrt_pd.rrt_step_eff_dt == forecast_date)[0]:                
                    if rrt_pd.rrt_step_type_num[j] in [252,452]:
                        if nws_pd.nws_stat[0] == 'W' and swf_exists:
                            current_plus_7days = getFutureDays(rrt_pd.rrt_step_eff_dt[j],swf_range)
                            seasonal_wf = getCalcSeasonalWF(current_plus_7days,
                                                            ptg_pd,ptg_exists,gr,gc,
                                                            dtp_pd,dtp_exists,pr,pc,
                                                            swf_pd,swf_exists,fr,fc)
                        else:
                            seasonal_wf = 1.0
                        base_nws = float(rrt_pd.rrt_step_volume[j]) / seasonal_wf
                        if rrt_pd.rrt_step_type_num[j] == 252:
                            vol_reset = True
                    
                    if rrt_pd.rrt_step_type_num[j] in [251,451,351]:
                        reset_factor = 1 + rrt_pd.rrt_step_uplift[j] / 100.0
                        total_reset_factor = total_reset_factor * reset_factor
                        if rrt_pd.rrt_step_type_num[j] == 251:
                            per_reset = True

            base_nws = base_nws * total_reset_factor
            base_nws_reset = base_nws  
            
            if rst_exists and dst_exists:
                # Now that the base is reset, we need to update steps if any, with the same reset factor
                for k in  np.where((rst_pd.rst_step_eff_dt < forecast_date) 
                                   & (rst_pd.rst_step_end_dt >= forecast_date))[0]:
                    if vol_reset:
                        rst_pd.rst_actual_nws[k] = base_nws
                    if per_reset:
                        if pd.notnull(rst_pd.rst_actual_nws[k]):
                            rst_pd.rst_actual_nws[k] = float(rst_pd.rst_actual_nws[k]) * total_reset_factor


                dst1_list =  dst_list[ (dst_list[:,0] <= forecast_date) & ( dst_list[:,1] >= forecast_date)]
                dst1_list = dst1_list[dst1_list[:,4].argsort()]
                
                if dst1_list.size > 0:
                    (tr,tc,dst1_pd,dst_exists) = getDailyAttributes(dst1_list if dst1_list is not None else [])
                    for m in range(tr):
                        for n in np.where((dst1_pd.dst_step_id[m] == rst_pd.rst_step_id) &
                                          (dst1_pd.dst_underlying_step_id[m] == rst_pd.rst_underlying_step_id))[0]:
                            if pd.notnull(rst_pd.rst_actual_nws[n]):
                                base_nws = rst_pd.rst_actual_nws[n]
                                rst_pd.rst_actual_nws[n] = base_nws
                                
                            else:
                                if pd.notnull(rst_pd.rst_step_volume[n]):
                                    if rst_pd.rst_continue_nws[n] == 'Y':
                                        prev_promo_end_dt = rst_pd.rst_step_eff_dt[n] + timedelta(days=-1)
                                        (base_nws,b2b_promo_found) = getB2BPromo(rst_pd,sr,prev_promo_end_dt)
                                    if not b2b_promo_found:
                                        seasonal_wf = 1.0
                                        if nws_pd.nws_stat[0] == 'W':
                                            if rst_pd.rst_wthr_sens_flag[n] in ['Y','A'] and swf_exists:
                                                current_plus_7days = getFutureDays(rst_pd.rst_step_eff_dt[n],swf_range)
                                                seasonal_wf = getCalcSeasonalWF(current_plus_7days,
                                                                                ptg_pd,ptg_exists,gr,gc,
                                                                                dtp_pd,dtp_exists,pr,pc,
                                                                                swf_pd,swf_exists,fr,fc)
                                            if rst_pd.rst_wthr_sens_flag[n] == 'N' and wef_exists:
                                                seasonal_wf = getCalcRealWF(rst_pd.rst_step_eff_dt[n],swf_range,
                                                                            ptg_pd,ptg_exists,gr,gc,
                                                                            dtp_pd,dtp_exists,pr,pc,
                                                                            wef_pd,wef_exists,wr,wc,
                                                                            swf_pd,swf_exists,fr,fc)
                                        base_nws = rst_pd.rst_step_volume[n] / seasonal_wf
                                    if pd.isnull(rst_pd.rst_actual_nws[n]):
                                        rst_pd.rst_actual_nws[n] = base_nws
                                else:
                                    if pd.notnull(rst_pd.rst_step_uplift[n]):
                                        base_nws = base_nws * (1+(rst_pd.rst_step_uplift[n])/100.0)        
                                        

            # Now that we have the forecast nws, we will find the respective ptp/ptg that needs to be applied
            # use the respective year week from the date_range broadcasted value
            
            days_ptp = 14.3    # default PTP, in case we end up not finding PTP/PTG entries
            if ptg_exists:
                # Use PTG, if available
                (days_ptp,ptg_found) = getDaysPtg(gr,ptg_pd,date_range[day])
            
            if not ptg_found:
                if dtp_exists:
                    # If there exists no PTG, use PTP
                    days_ptp = getDaysPtp(pr,dtp_pd,date_range[day])
            
            # Derive the weather factors for the weather products
            if nws_pd.nws_stat[0] == 'W' and wef_exists:
                days_wthr_factor = getRealWF(forecast_date,wef_pd,wr,swf_pd,fr,date_range[day])
            else:
                days_wthr_factor = 1.0

            exp_daily_sls = np.around(base_nws*(days_ptp/100.0)*days_wthr_factor,2)

            # The EDS for the day along with the respective attributes needs to appended to output list
            # Note that Pyspark DF's doesn't support numpy float type. Hence there is a need to convert them to float
            # before returning
            forecasted_date = forecast_date.strftime("%Y-%m-%d")
            forecast_data.extend([current_date,nws_pd.sg_cd[0],forecasted_date,float(base_nws),nws_pd.nws_stat[0],
                                  float(np.around(days_ptp,2)),float(days_wthr_factor),float(exp_daily_sls)]) 
            
        else:
            # The product is not stocked, default the forecast to zeros.
            forecasted_date = forecast_date.strftime("%Y-%m-%d")
            forecast_data.extend([current_date,nws_pd.sg_cd[0],forecasted_date,0.0,' ',0.0,1.0,0.0]) 
    return forecast_data

In [69]:
# Mid-term forecast Runner
t1_comp = datetime.now()

#intermediate_output = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/intermediate_output_allStores_F11MA_2017-08-02_merged_version').filter('nws_key.ro_no=5777').filter('nws_key.bpr_tpn=50935010')
#intermediate_output = intermediate_output.repartition(200000)
intermediate_output = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/intermediate_output_allStores_F11MA_2017-08-02_merged_version').filter('nws_key.ro_no=4483').filter('nws_key.bpr_tpn=50935010')

# Broadcast the CAL date range required for get_forecast function.
cal_sel_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/cal_bc_set1_'+sub_group)
cal_list1 = cal_sel_data.rdd.map(lambda x : (x.calendar_date,(x.year_week,x.week_number,x.day_number))).collectAsMap()
cal_bc_var1 = sc.broadcast(cal_list1)

cal_sel_data = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/cal_bc_set2_'+sub_group)
cal_list2 = cal_sel_data.rdd.map(lambda x : (x.calendar_date,(x.year_week,x.week_number,x.day_number))).collectAsMap()
cal_bc_var2 = sc.broadcast(cal_list2)


forecast_rdd = intermediate_output.map(getForecast)
#forecast_rdd.take(10)

# # This is meant to de-normalise the output
number_of_columns = 8 # number of columns for each store/product
starting_point = 1 # in python terms, so the counter starts from 0
forecast_at_day = (forecast_rdd.flatMap(lambda x:( ((x[:starting_point+1]) + v) for  v in [x[starting_point:][i:i + number_of_columns] for i in xrange(starting_point, len(x)-starting_point, number_of_columns)]
                                )  
                      )
             .toDF(schema=forecast_data_schema())
          )

path_name = ('/insight_labs/rdf/output/cr_forecast/final_output_'+sub_group+'_'+current_date+'_'+'merged_version')
forecast_at_day.coalesce(2000).write.format("orc").save(path=path_name,mode="overwrite")


# print 'has been calculated'
t2_comp = datetime.now()
print t2_comp - t1_comp

0:00:33.844580


In [70]:
#intermediate_output = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/intermediate_output_allStores_F11MA_2017-08-02_merged_version')
intermediate_output = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/intermediate_output_allStores_'+sub_group+'_'+current_date+'_'+'merged_version')
#.limit(18000000)
#intermediate_output = sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/intermediate_output_allStores_allSG_2017-08-08_merged_version').filter('nws_key.ro_no=2002')
intermediate_output = intermediate_output.repartition(200000)
#.filter('nws_key.ro_no=5288').filter('nws_key.bpr_tpn=50935010')
#intermediate_output.cache()
#intermediate_output.count()

In [71]:

path=('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/intermediate_output_allStores_'+sub_group+'_'+current_date+'_'+'merged_version')
print path

hdfs://pphdp/insight_labs/rdf/output/cr_forecast/intermediate_output_allStores_F11MA_2017-10-15_merged_version


In [72]:
df=sqlContext.read.orc('hdfs://pphdp/insight_labs/rdf/output/cr_forecast/final_output_'+sub_group+'_'+current_date+'_'+'merged_version')
#df.show(200)

In [74]:
df.show(10)

+--------------------+-------------------+----------+--------------+-------------+------------+--------+---------+------------------+--------------------+
|Retail_Outlet_Number|Base_Product_Number|  Run_Date|Sub_Group_Code|Forecast_Date|Forecast_NWS|NWS_Stat|Daily_PTP|    Weather_Factor|Expected_Daily_Sales|
+--------------------+-------------------+----------+--------------+-------------+------------+--------+---------+------------------+--------------------+
|                4483|           50935010|2017-10-15|         F11MA|   2017-08-18|      119.16|       W|     17.9|1.1597141027450562|               24.74|
|                4483|           50935010|2017-10-15|         F11MA|   2017-08-19|      119.16|       W|     18.6|1.1737021207809448|               26.01|
|                4483|           50935010|2017-10-15|         F11MA|   2017-08-20|      119.16|       W|     11.1|1.1426117420196533|               15.11|
|                4483|           50935010|2017-10-15|         F11MA|  

In [ ]:
# url = "jdbc:teradata://TDPS.ukroi.tesco.org/DATABASE=DXWI_PROD_RDF_PLAY_PEN"
# properties = {
#     "user": "xg66",
#     "password": "tesco123",
#      "driver" : "com.teradata.jdbc.TeraDriver"
# }
# mode="overwrite"_merged_version
#forecast_at_day.coalesce(1).write.jdbc(url=url, table="MID_TERM_FORECAST", mode=mode, properties=properties)
## vijaydf.write.format('com.databricks.spark.csv').save(path='/insight_labs/rdf/output/cr_forecast/final_output_'+sub_group+'_CSV_merged_version',mode='overwrite')